<a href="https://colab.research.google.com/github/ShakutaiGit/Final_Project_Applied_Data_science/blob/main/Improvement_of_source_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

relevant imports 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model 
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
import numpy as np
import pandas as pd 
!pip install optuna
import optuna
from sklearn.model_selection import KFold
from skimage.transform import resize
from skimage.transform import rescale
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
import cv2
from statistics import mean
from google.colab import files
import time 
import csv


     |████████████████████████████████| 307kB 8.5MB/s 
     |████████████████████████████████| 174kB 51.4MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 143kB 37.9MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 112kB 53.8MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=62ff5c9b978445ed8ab04a845bb977463ae568704c38f3e2a07639555714b20a
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


global variables

In [2]:
optimizer = None
size = 128 
batch_size = 128 # like the original paper 
classes = 0
AUTOTUNE = tf.data.AUTOTUNE
x_train = []
x_test = []
y_train = []
y_test = []
algorithem_name = 'Improvement'
database_name = None

loading the dataset and pre processing 

In [5]:
train_ds ,train_info= tfds.load('oxford_flowers102', split='train[:80%]', with_info=True)
# val_ds,val_info = tfds.load('oxford_flowers102', split='train[80%:90%]', with_info=True)
test_ds , test_info = tfds.load('oxford_flowers102', split='train[90%:]', with_info=True)
## pre processing 
database_name = train_info.name
resize_and_rescale = tf.keras.Sequential([
layers.experimental.preprocessing.Resizing(size, size),
layers.experimental.preprocessing.Rescaling(1./255)
])

def pre_process(data):
  pictures=[]
  labels = []
  for i in data:
    temp = resize_and_rescale(i['image'])
    temp = np.asarray(temp)
    pictures.append(temp)
    labels.append(int(i['label']))
  return np.asarray(pictures),np.asarray(labels)

x_train,y_train =pre_process(train_ds)
x_test,y_test =pre_process(test_ds)
classes = train_info.features['label'].num_classes

In [6]:
def creating_the_model():
  inputs = layers.Input(shape=(size, size, 3))
  pre_trained_model = keras.applications.ResNet50V2(include_top=False, weights='imagenet',input_tensor=inputs,pooling='avg')
  pre_trained_model.trainable= False
  
  x = layers.BatchNormalization()(pre_trained_model.output)
  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(102, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="ResNet")
  #unfreeze some layers of the pretrained model
  for layer in model.layers[-10:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
  return model



In [7]:
def objective(trial,outer_x_train,outer_y_train):
  lr = trial.suggest_float(name="lr", low = 0.00001,high= 0.1, log=True)
  steps = trial.suggest_float(name="steps", low = 1,high=1000)
  t_mult = trial.suggest_categorical(name="t_mult", choices =[1,2])
  m_mult = trial.suggest_float(name="m_mult", low=0,high=1)
 
  

  internal_kfold = KFold(n_splits=3, shuffle=True)
  fold_acc = []  
  for inner_train_index,inner_test_index in internal_kfold.split(outer_x_train,outer_y_train):
    inner_x_train, inner_x_test = outer_x_train[inner_train_index], outer_x_train[inner_test_index]
    inner_y_train, inner_y_test = outer_y_train[inner_train_index], outer_y_train[inner_test_index]
    model = creating_the_model()
    model.compile(
    loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(
  initial_learning_rate=trial.suggest_float(name="lr", low = 0.00001,high= 0.1, log=True),
  first_decay_steps=trial.suggest_float(name="steps", low = 1,high=1000), 
  t_mul=trial.suggest_categorical(name="t_mult", choices =[1,2]), 
  m_mul=trial.suggest_float(name="m_mult", low=0,high=1), 
  alpha=0.0)
      ), metrics=["accuracy"]
  )
    model.fit(
        x=inner_x_train,
        y=to_categorical(inner_y_train,num_classes=classes),
        # validation_data= (inner_x_test,to_categorical(,inner_y_test)),
        # shuffle=True,
        validation_split=0.1,
        batch_size=batch_size,
        epochs=5
    )
    score = model.evaluate(inner_x_test,to_categorical(inner_y_test, num_classes=classes) , verbose=0)
    fold_acc.append(score[1] * 100)
  return mean(fold_acc)

In [8]:
def fold_dict_creator(Hyper_Parameters_Values,score,training_time,inference_time,fold):
  fold_dict = {}
  fold_dict['Dataset Name'] = database_name
  fold_dict['Algorithm Name'] = algorithem_name
  fold_dict['Cross Validation [1-10]'] = fold
  fold_dict['Hyper-Parameters Values'] = Hyper_Parameters_Values
  fold_dict['Accuracy'] = score[1]
  fold_dict['TPR'] = score[2]
  fold_dict['FPR'] = score[3]
  fold_dict['Precision'] = score[4]
  fold_dict['AUC'] = score[5]
  fold_dict['PR-Curve'] = score[6]
  fold_dict['Training Time'] = training_time
  fold_dict['Inference Time'] = inference_time
  return fold_dict


In [9]:
outer_kfold = KFold(n_splits=10, shuffle=True)
fold_num = 0
dict_data= []
for outer_train_index,outer_test_index in outer_kfold.split(x_train, y_train):
  fold_num += 1
  outer_x_train, outer_x_test = x_train[outer_train_index], x_train[outer_test_index]
  outer_y_train, outer_y_test = y_train[outer_train_index], y_train[outer_test_index]
  study = optuna.create_study(direction="maximize")
  study.optimize(lambda trail:objective(trail,outer_x_train,outer_y_train), n_trials=10)
  model = creating_the_model()
  model.compile(
    loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(
        learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(
  initial_learning_rate=study.best_params["lr"],
  first_decay_steps=study.best_params["steps"], 
  t_mul=study.best_params["t_mult"], 
  m_mul=study.best_params["m_mult"], 
  alpha=0.0)
      ), metrics=["accuracy", tf.keras.metrics.SensitivityAtSpecificity(0.5),
      tf.keras.metrics.SpecificityAtSensitivity(0.5), 
      tf.keras.metrics.Precision(),
      tf.keras.metrics.AUC(),
      tf.keras.metrics.AUC(curve = 'pr')]
  )
  start_time = time.time()
  model_info = model.fit(
    outer_x_train,
    to_categorical(outer_y_train,num_classes=classes),
    validation_split= 0.15,
    shuffle=True,
    batch_size=batch_size,
    epochs=5,
  )
  training_time = time.time() - start_time
  start_time = time.time()
  score = model.evaluate(outer_x_test,to_categorical(outer_y_test,num_classes=classes))
  print(model.metrics_names)
  inference_time = time.time() - start_time
  dict_data.append(fold_dict_creator(str(study.best_params), score, training_time , inference_time, fold_num))
csv_file = "results_algo_{}_data_set{}.csv".format(algorithem_name,database_name)
with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=dict_data[0].keys())
    writer.writeheader()
    for data in dict_data:
        writer.writerow(data)
files.download(csv_file)  

[I 2021-07-15 11:47:39,822] A new study created in memory with name: no-name-30404a45-60ab-4f1b-bd81-25c8a28be4be


Epoch 1/5
4/4 [==============================] - 10s 1s/step - loss: 5.5866 - accuracy: 0.0318 - val_loss: 5.0941 - val_accuracy: 0.0204
Epoch 2/5
4/4 [==============================] - 1s 131ms/step - loss: 3.0487 - accuracy: 0.2864 - val_loss: 4.4792 - val_accuracy: 0.0612
Epoch 3/5
4/4 [==============================] - 0s 125ms/step - loss: 1.8461 - accuracy: 0.6023 - val_loss: 4.1948 - val_accuracy: 0.0816
Epoch 4/5
4/4 [==============================] - 0s 124ms/step - loss: 1.1514 - accuracy: 0.7909 - val_loss: 4.0693 - val_accuracy: 0.1224
Epoch 5/5
4/4 [==============================] - 0s 120ms/step - loss: 0.7279 - accuracy: 0.8864 - val_loss: 3.9911 - val_accuracy: 0.1020
Epoch 1/5
4/4 [==============================] - 4s 372ms/step - loss: 5.5526 - accuracy: 0.0295 - val_loss: 4.8128 - val_accuracy: 0.0816
Epoch 2/5
4/4 [==============================] - 0s 124ms/step - loss: 3.2465 - accuracy: 0.2568 - val_loss: 4.2363 - val_accuracy: 0.1837
Epoch 3/5
4/4 [==============

[I 2021-07-15 11:48:11,517] Trial 0 finished with value: 24.24278010924657 and parameters: {'lr': 0.0005136845085760202, 'steps': 179.3601111912019, 't_mult': 2, 'm_mult': 0.08236007996408745}. Best is trial 0 with value: 24.24278010924657.


Epoch 1/5
4/4 [==============================] - 4s 378ms/step - loss: 5.0580 - accuracy: 0.0909 - val_loss: 7.0621 - val_accuracy: 0.1224
Epoch 2/5
4/4 [==============================] - 0s 124ms/step - loss: 1.0903 - accuracy: 0.7205 - val_loss: 10.3407 - val_accuracy: 0.1429
Epoch 3/5
4/4 [==============================] - 0s 124ms/step - loss: 0.4465 - accuracy: 0.9068 - val_loss: 11.0139 - val_accuracy: 0.1837
Epoch 4/5
4/4 [==============================] - 0s 123ms/step - loss: 0.2436 - accuracy: 0.9591 - val_loss: 11.5981 - val_accuracy: 0.2245
Epoch 5/5
4/4 [==============================] - 0s 120ms/step - loss: 0.1078 - accuracy: 0.9773 - val_loss: 12.5135 - val_accuracy: 0.2041
Epoch 1/5
4/4 [==============================] - 4s 375ms/step - loss: 4.9027 - accuracy: 0.0932 - val_loss: 7.4223 - val_accuracy: 0.1429
Epoch 2/5
4/4 [==============================] - 0s 124ms/step - loss: 1.1602 - accuracy: 0.7318 - val_loss: 11.4610 - val_accuracy: 0.0816
Epoch 3/5
4/4 [=======

[I 2021-07-15 11:48:34,960] Trial 1 finished with value: 22.20976948738098 and parameters: {'lr': 0.002697832657247236, 'steps': 780.7734025365836, 't_mult': 2, 'm_mult': 0.1438569598663324}. Best is trial 0 with value: 24.24278010924657.


...

Epoch 1/5
4/4 [==============================] - 5s 401ms/step - loss: 5.4488 - accuracy: 0.0272 - val_loss: 4.8472 - val_accuracy: 0.0204
Epoch 2/5
4/4 [==============================] - 1s 132ms/step - loss: 2.8556 - accuracy: 0.3469 - val_loss: 4.1903 - val_accuracy: 0.1224
Epoch 3/5
4/4 [==============================] - 1s 137ms/step - loss: 1.6957 - accuracy: 0.6281 - val_loss: 3.8861 - val_accuracy: 0.2245
Epoch 4/5
4/4 [==============================] - 1s 130ms/step - loss: 0.9370 - accuracy: 0.8413 - val_loss: 3.7216 - val_accuracy: 0.2245
Epoch 5/5
4/4 [==============================] - 1s 129ms/step - loss: 0.5924 - accuracy: 0.9161 - val_loss: 3.6018 - val_accuracy: 0.2449
Epoch 1/5
4/4 [==============================] - 4s 383ms/step - loss: 5.5840 - accuracy: 0.0159 - val_loss: 5.0076 - val_accuracy: 0.0612
Epoch 2/5
4/4 [==============================] - 1s 130ms/step - loss: 2.9016 - accuracy: 0.3356 - val_loss: 4.4765 - val_accuracy: 0.0816
Epoch 3/5
4/4 [============

[I 2021-07-15 12:31:52,622] Trial 9 finished with value: 25.850340723991394 and parameters: {'lr': 0.0006149785235683608, 'steps': 343.17048555381825, 't_mult': 2, 'm_mult': 0.6471125581036888}. Best is trial 9 with value: 25.850340723991394.


Epoch 1/5
5/5 [==============================] - 7s 552ms/step - loss: 6.1350 - accuracy: 0.0064 - sensitivity_at_specificity_9: 0.3830 - specificity_at_sensitivity_9: 0.0000e+00 - precision_9: 0.0294 - auc_18: 0.4958 - auc_19: 0.0103 - val_loss: 5.8305 - val_accuracy: 0.0000e+00 - val_sensitivity_at_specificity_9: 0.4324 - val_specificity_at_sensitivity_9: 0.0000e+00 - val_precision_9: 0.0000e+00 - val_auc_18: 0.4926 - val_auc_19: 0.0093
Epoch 2/5
5/5 [==============================] - 1s 167ms/step - loss: 5.9585 - accuracy: 0.0208 - sensitivity_at_specificity_9: 0.4183 - specificity_at_sensitivity_9: 0.0000e+00 - precision_9: 0.0000e+00 - auc_18: 0.5131 - auc_19: 0.0108 - val_loss: 5.6354 - val_accuracy: 0.0000e+00 - val_sensitivity_at_specificity_9: 0.4775 - val_specificity_at_sensitivity_9: 0.0000e+00 - val_precision_9: 0.0000e+00 - val_auc_18: 0.4929 - val_auc_19: 0.0093
Epoch 3/5
5/5 [==============================] - 1s 164ms/step - loss: 5.9176 - accuracy: 0.0144 - sensitivity

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import csv
csv_file = "results_algo_{}_data_set{}.csv".format(algorithem_name,database_name)
with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=dict_data[0].keys())
    writer.writeheader()
    for data in dict_data:
        writer.writerow(data)
files.download(csv_file)  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>